In [ ]:
https://medium.com/@kshitiz.sahay26/fine-tuning-llama-2-for-news-category-prediction-a-step-by-step-comprehensive-guide-to-fine-tuning-48c06dee28a9

In [ ]:
# # Load fine-tuned weights
# model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map = "auto", torch_dtype = torch.bfloat16)
# # Merge the LoRA layers with the base model
# model = model.merge_and_unload()

# # Save fine-tuned model at a new location
# output_merged_dir = "results/news_classification_llama2_7b/final_merged_checkpoint"
# os.makedirs(output_merged_dir, exist_ok = True)
# model.save_pretrained(output_merged_dir, safe_serialization = True)

# # Save tokenizer for easy inference
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.save_pretrained(output_merged_dir)

# # Fine-tuned model name on Hugging Face Hub
# new_model = "sahayk/news-classification-18-llama-2-7b"

# # Push fine-tuned model and tokenizer to Hugging Face Hub
# model.push_to_hub(new_model, use_auth_token = True)
# tokenizer.push_to_hub(new_model, use_auth_token = True)

In [ ]:
%pip install accelerate==0.26.1 peft==0.8.2 bitsandbytes==0.42.0 transformers==4.37.2 trl==0.7.10

In [1]:
import torch
# GPU 사용 가능 -> True, GPU 사용 불가 -> False
print(torch.cuda.is_available())


True


In [2]:
from transformers import AutoTokenizer, LlamaForCausalLM
import torch
import argparse
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 라마2 허깅페이스 모델 불러오기
base_model = "beomi/llama-2-ko-7b"

compute_dtype = getattr(torch, 'float16')

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 어댑터 추가
model = PeftModel.from_pretrained(model, "./hkcode_kor")

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00015.safetensors:   0%|          | 0.00/919M [00:00<?, ?B/s]

model-00002-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00003-of-00015.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

ChunkedEncodingError: ('Connection broken: IncompleteRead(432118510 bytes read, 534721066 more expected)', IncompleteRead(432118510 bytes read, 534721066 more expected))

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "한국폴리텍대학 스마트금융과는 어디에 위치하나요?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "한국폴리텍대학 스마트금융과 면접시에는 어떤걸 준비하고 가면 될까요?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

### 챗봇연결

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
# !pip install --upgrade gradio
!pip install -q gradio==3.45.0
!pip install -q gradio --use-deprecated=legacy-resolver
!pip install typing_extensions --upgrade
import gradio as gr

In [ ]:
llama_pipeline = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [ ]:
SYSTEM_PROMPT = """<s>[INST] <<SYS>>
You are a helpful bot. Your answers are clear and concise.
<</SYS>>

"""

# Formatting function for message and history
def format_message(message: str, history: list, memory_limit: int = 3) -> str:
    """
    Formats the message and history for the Llama model.

    Parameters:
        message (str): Current message to send.
        history (list): Past conversation history.
        memory_limit (int): Limit on how many past interactions to consider.

    Returns:
        str: Formatted message string
    """
    # always keep len(history) <= memory_limit
    if len(history) > memory_limit:
        history = history[-memory_limit:]

    if len(history) == 0:
        return SYSTEM_PROMPT + f"{message} [/INST]"

    formatted_message = SYSTEM_PROMPT + f"{history[0][0]} [/INST] {history[0][1]} </s>"

    # Handle conversation history
    for user_msg, model_answer in history[1:]:
        formatted_message += f"<s>[INST] {user_msg} [/INST] {model_answer} </s>"

    # Handle the current message
    formatted_message += f"<s>[INST] {message} [/INST]"

    return formatted_message

In [ ]:
# Generate a response from the Llama model
def get_llama_response(message: str, history: list) -> str:
    """
    Generates a conversational response from the Llama model.

    Parameters:
        message (str): User's input message.
        history (list): Past conversation history.

    Returns:
        str: Generated response from the Llama model.
    """
    query = format_message(message, history)
    response = ""

    sequences = llama_pipeline(
        query,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=1024,
    )

    generated_text = sequences[0]['generated_text']
    response = generated_text[len(query):]  # Remove the prompt from the output

    print("Chatbot:", response.strip())
    return response.strip()

In [ ]:
gr.ChatInterface(get_llama_response).launch()